In [30]:
import os 
os.chdir('c:\\Users\\gupta\\OneDrive\\Documents\\Medical-Help-ChatBot')

In [31]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [32]:
def load_pdf(data):
    loader=DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents


In [33]:
extractedData= load_pdf(data='Data/')

In [34]:
def text_splitter(documents):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts=text_splitter.split_documents(documents)
    return texts

In [35]:
text_chunks=text_splitter(extractedData)
print(f"Number of chunks: {len(text_chunks)}")

Number of chunks: 7020


In [36]:
from langchain.embeddings import HuggingFaceEmbeddings

In [37]:
def download_huggingface_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [38]:
embeddings=download_huggingface_embeddings()

C:\Users\gupta\AppData\Local\Temp\ipykernel_12468\2743064051.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [39]:
query_result=embeddings.embed_query("Hello")
print("length", len(query_result))

length 384


In [49]:
from dotenv import load_dotenv
load_dotenv()
pinecone_api_key=os.environ.get("PINECONE_API_KEY")
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")
os.environ["PINECONE_API_KEY"]=pinecone_api_key
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="medicalbot",
    embedding=embeddings,
)


In [42]:
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embeddings,
)

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", #"gemini-1.5-pro"
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. Use three sentences maximum and keep the "
    "answer concise. "
    "If you don't know the answer,just say that you don't know "
    "in one line."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])